In [10]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pandas import DataFrame
import pandas as pd
import sys

def progress(count, total, suffix=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '#' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('|%s| %s%s ...%s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()

def get_soup(url):
    html_doc = urlopen(url).read()
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup

# 승용차 정보 크롤링 함수 수정
def my_car_price(url):
    soup_wt = get_soup(url)
    info = []

    try:
        table_body = soup_wt.find("table", class_="cyber").find("tbody")
        if not table_body:
            raise AttributeError("Table body not found.")
        
        num_carth = len(table_body.find_all("a", class_="sub"))
    except AttributeError as e:
        print(f"Error at page {url}: {e}")
        return DataFrame(info)

    for i in range(num_carth):
        try:
            car_url = str('https://www.bobaedream.co.kr') + table_body.find_all("a", class_="sub")[i]['href']
            car_soup = get_soup(car_url)
            sell_price_tag = car_soup.find("div", class_="information").find("em", class_="cr")
            if sell_price_tag:
                sell_price = sell_price_tag.get_text().replace("만원", "").replace(",", "")
                info.append({'b.sell_price(만원)': sell_price})
        except Exception as e:
            print(f"Error at page {url}: {e}")

    return DataFrame(info)

def concatthing(totpage, base_url):
    appended_data = []
    for k in range(totpage):
        try:
            progress(k + 1, totpage, suffix='crawling car page')
            url = f"{base_url}&page={k + 1}"
            dfpg = my_car_price(url)
            appended_data.append(dfpg)
        except Exception as e:
            print(f"Error at page {url}: {e}")

    if not appended_data:
        print("No data to concatenate.")
        return None

    appended_data = pd.concat(appended_data).reset_index(drop=True)
    return appended_data

# 실행할 때 사용하는 코드
base_url = 'https://www.bobaedream.co.kr/dealguide/market_price_new.php?gubun=K&maker_no=49&group_no=5&model_no%5B%5D=2107&view_size=20'
total_pages = 2

used_car_price = concatthing(total_pages, base_url)

if used_car_price is not None:
    print(used_car_price)

    # 결과를 CSV 파일로 저장
    used_car_price.to_csv('used_car_price.csv', index=False)


Error at page https://www.bobaedream.co.kr/dealguide/market_price_new.php?gubun=K&maker_no=49&group_no=5&model_no%5B%5D=2107&view_size=20&page=1: 'NoneType' object has no attribute 'find'
Error at page https://www.bobaedream.co.kr/dealguide/market_price_new.php?gubun=K&maker_no=49&group_no=5&model_no%5B%5D=2107&view_size=20&page=2: 'NoneType' object has no attribute 'find'
Empty DataFrame
Columns: []
Index: []
